In [1]:
# Install the required libraries in your Google Colab environment
!pip install gymnasium stable-baselines3 highway-env -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 121.1 MB/s eta 0:00:00


In [2]:
# Import the necessary libraries
import gymnasium as gym
import highway_env
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [3]:
# THE ENVIRONMENT
# Step 1.1: Choose the Environment
# Initialize the environment.
env = gym.make('highway-v0',config={"vehicles_count":50})

In [4]:
from stable_baselines3 import PPO
from google.colab import drive
from google.colab import data_table

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Step A.6.2: Load the all the saved PPO model
model = PPO.load('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/01_trained_models/6_ppo_highway_biased_hf_lrs_conservative')         # Update directory location 1

Trajectory Collection with Additional information (Collision Flag and Lane Index)

In [6]:
# TRAJECTORY COLLECTION WITH ADDTIONNAL INFORMATION
# Initialize a list to store trajectory data
trajectories = []

# FUNCTION TO COLLECT TRAJECTORY DATA (state-action-reward transitions).

def collect_trajectory_data(env, model, num_episodes,seed):
    """
    Collect trajectory data for a number of episodes.
    Each trajectory contains state-action-reward sequences.
    """
    trajectory_data = []

    for episode in range(num_episodes):
        state, _ = env.reset(seed=seed)  # Reset the environment at the start of each episode              #  change environment name here
        done = False
        episode_data = []

        while not done:
            # Get action from the trained PPO model
            action, _states = model.predict(state, deterministic = True)                                       # change model name here

            # Take the action and get next state and reward
            next_state, reward, terminated, truncated, info = env.step(action)                    #  change environment name here
            done = terminated or truncated
            # Extract lane index and collision flag
            lane_index = int(env.unwrapped.vehicle.lane_index[2])
            collision_flag = int(info.get('crashed', 0))

            # Store the trajectory: (state, action, reward, next_state)
            episode_data.append({
                "state": state,
                "action": action,
                "reward": reward,
                "next_state": next_state,
                "lane_indices": lane_index,
                "collision_flags": collision_flag
            })

            # Update the state for the next iteration
            state = next_state

        # Add the episode data to the overall trajectory list
        trajectory_data.append(episode_data)

    return trajectory_data

In [7]:
# FUNCTION TO PREPROCESS TRAJECTORY DATA
def preprocess_trajectory_data(trajectory_data):
    """
    Preprocesses the trajectory data into a structured format for further analysis.
    Returns a DataFrame with columns: episode, time_step, state, action, reward, next_state, speed, and reward_details.
    """
    processed_data = []

    for episode_num, episode_data in enumerate(trajectory_data):
        for time_step, step in enumerate(episode_data):
            # Flatten the state and next_state for easy interpretation (if they are multi-dimensional)
            state = np.array(step['state']).flatten()  # Flatten the state vector (if multi-dimensional)
            next_state = np.array(step['next_state']).flatten()  # Flatten the next_state vector

            collision_flag = step['collision_flags']
            lane_index = step['lane_indices']

            # Append the processed data for this step
            processed_data.append({
                "episode": episode_num,
                "time_step": time_step,
                "state": state,
                "action": step['action'],
                "reward": step['reward'],
                "next_state": next_state,
                "collision_flag": collision_flag,
                "lane_index": lane_index
            })

    # Convert the list of processed data into a DataFrame
    df = pd.DataFrame(processed_data)
    return df

0. GENERATING TRAJECTORIES FOR TESTING (HUMAN FEEDBACK DIRECT)

First Human feedback direct data frame

In [8]:
# Collect data for 100 episodes
trajectory_data_1 = collect_trajectory_data(env, model, num_episodes=100,seed=2)

In [9]:
# Preprocess the trajectory data
trajectory_df_seed_1 = preprocess_trajectory_data(trajectory_data_1)

In [10]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_seed_1)

,episode,time_step,state,action,reward,next_state,collision_flag,lane_index
0,0,0,"[1.0, 0.888621, 0.75, 0.3125, 0.0, 1.0, 0.0987...",2,0.866667,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.089255214...",0,3
1,0,1,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.089255214...",2,0.866667,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.07708404,...",0,3
2,0,2,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.07708404,...",2,0.866667,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.063126676...",0,3
3,0,3,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.063126676...",2,0.866667,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.047992706...",0,3
4,0,4,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.047992706...",2,0.866667,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, 0.032074265...",0,3
...,...,...,...,...,...,...,...,...
1195,99,7,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, -0.00122240...",2,0.866667,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, -0.01835671...",0,3
1196,99,8,"[1.0, 1.0, 0.75, 0.3125, 0.0, 1.0, -0.01835671...",4,0.756118,"[1.0, 1.0, 0.75, 0.26068053, 0.0, 1.0, 0.01649...",0,3
1197,99,9,"[1.0, 1.0, 0.75, 0.26068053, 0.0, 1.0, 0.01649...",4,0.737227,"[1.0, 1.0, 0.75, 0.25182518, 0.0, 1.0, 0.01623...",0,3
1198,99,10,"[1.0, 1.0, 0.75, 0.25182518, 0.0, 1.0, 0.01623...",4,0.733999,"[1.0, 1.0, 0.75, 0.2503119, 0.0, 1.0, -0.01016...",0,3


In [11]:
# Save the dataframe as a pickle file
trajectory_df_seed_1.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/6_biased_hf_lrs_conservative/1_biased_hf_lrs_conservative_df.pkl')   # Update directory location 2


Second Human feedback direct data frame


In [12]:
# Collect data for 100 episodes
trajectory_data_2 = collect_trajectory_data(env, model, num_episodes=100,seed=10)

In [13]:
# Preprocess the trajectory data
trajectory_df_seed_2 = preprocess_trajectory_data(trajectory_data_2)

In [14]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_seed_2)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
# Save the dataframe as a pickle file
trajectory_df_seed_2.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/6_biased_hf_lrs_conservative/2_biased_hf_lrs_conservative_df.pkl')   # Update directory location 3

Third Human feedback direct data frame

In [16]:
# Collect data for 100 episodes
trajectory_data_3 = collect_trajectory_data(env, model, num_episodes=100,seed=6)

/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/observation.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = utils.lmap(df[feature], [f_range[0], f_range[1]], [-1, 1])
/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/observation.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = np.clip(df[feature], -1, 1)
/usr/local/lib/python3.11/dist-packages/highway_env/envs/common/observation.py:228: SettingWithCopyWarning: 
A value is trying to be set 

In [17]:
# Preprocess the trajectory data
trajectory_df_seed_3 = preprocess_trajectory_data(trajectory_data_3)

In [18]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_seed_3)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# Save the dataframe as a pickle file
trajectory_df_seed_3.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/6_biased_hf_lrs_conservative/3_biased_hf_lrs_conservative_df.pkl')      # Update directory location 4

Fourth Human feedback direct data frame

In [20]:
# Collect data for 100 episodes
trajectory_data_4 = collect_trajectory_data(env, model, num_episodes=100,seed=20)

In [21]:
# Preprocess the trajectory data
trajectory_df_seed_4 = preprocess_trajectory_data(trajectory_data_4)

In [22]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_seed_4)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
# Save the dataframe as a pickle file
trajectory_df_seed_4.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/6_biased_hf_lrs_conservative/4_biased_hf_lrs_conservative_df.pkl')     # Update directory location 5

Fifth Human feedback direct data frame

In [24]:
# Collect data for 100 episodes
trajectory_data_5 = collect_trajectory_data(env, model, num_episodes=100,seed=34)

In [25]:
# Preprocess the trajectory data
trajectory_df_seed_5 = preprocess_trajectory_data(trajectory_data_5)

In [26]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_seed_5)

Output hidden; open in https://colab.research.google.com to view.

In [27]:
# Save the dataframe as a pickle file
trajectory_df_seed_5.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/03_test_trajectories/6_biased_hf_lrs_conservative/5_biased_hf_lrs_conservative_df.pkl')    # Update directory location 6